# 05. hdf5 파일에서 데이터 Sampling
- 작성 : 정민정 (https://github.com/jeina7)

---
- `hdf5`로 저장된 데이터셋을 batch로 추출하기 위한 `Sampler_crawled_data` 생성
---

In [1]:
import h5py

file_path = './common/data/news_sample.hdf5'
fi = h5py.File(file_path, 'r')

In [2]:
fi.keys()

<KeysViewHDF5 ['category']>

In [3]:
fi['category']

<HDF5 group "/category" (54 members)>

In [8]:
for key in fi['category'].keys():
    print(fi['category'][key][0].shape, fi['category'][key][1].shape)

(825,) (273,)
(444,) (295,)
(325,) (341,)
(781,) (420,)
(404,) (339,)
(461,) (633,)
(317,) (286,)
(540,) (414,)
(301,) (308,)
(267,) (292,)
(366,) (931,)
(288,) (304,)
(378,) (430,)
(400,) (468,)
(1240,) (1309,)
(445,) (257,)
(466,) (455,)
(607,) (483,)
(488,) (375,)
(316,) (282,)
(468,) (670,)
(441,) (554,)
(571,) (409,)
(493,) (320,)
(518,) (329,)
(477,) (580,)
(518,) (287,)
(384,) (525,)
(561,) (639,)
(425,) (647,)
(572,) (416,)
(1765,) (266,)
(498,) (268,)
(264,) (668,)
(266,) (264,)
(275,) (418,)
(717,) (275,)
(774,) (571,)
(940,) (633,)
(270,) (337,)
(321,) (694,)
(979,) (284,)
(369,) (461,)
(680,) (371,)
(595,) (270,)
(466,) (444,)
(504,) (406,)
(401,) (784,)
(614,) (356,)
(281,) (383,)
(414,) (382,)
(382,) (340,)
(261,) (360,)
(423,) (488,)


In [21]:
import os
from common.src.tokenization import *
from common.src.refine_punc import *

vocab_path = './common/models/345K/vocab.txt'
tokenizer = FullTokenizer(vocab_file=vocab_path, do_lower_case=False)
tokens = tokenizer.convert_ids_to_tokens(fi['category']['0'][0])
text = refine_punc(tokens)
text

'세계 각국 한인 여성 리더들이 모이는제14회 세계한민족여성네트워크: a s al 행사가 26일 개막했다. 행사는 여성가족부, 경상북도, 구미시, 세계한민족여성재단 등이 주최하며 29일까지 경상북도 구미시에서 열린다. 올해제14회 세계한민족여성네트워크에는 국내 280여 명, 해외 220 여명 등 세계 36개국 500여 명이 참가한다. 경력단절 예방 및 일· 가정 양립을 위한 글로벌 파트너십 강화를 주제로 국내외 경험과 지혜를 모아 실질적인 정책을 논의한다. 남성들의 육아휴직 활성화를 위한 아빠의 달 제도, 직장어린이집과 아이돌보미서비스, 육아휴직 등 국내· 외 다양한 정책을 소개하고 여성새로일하기센터 등 여성 재취업을 위한 발전방안에 대해서도 논의할 예정이다. 특히지난해일본군 위안부 피해자 문제 해결을 촉구하는 성명서 발표에 이어일본군 위안부 피해자 기록의 유네스코 등재 등 국제사회 공조를 얻어내기 위해 세계적 수준의 영향력을 강화하는 방안도 함께 논의한다. 행사 첫 날인 26일에는 20개 지역지역담당관 회의 등을 통해차세대 여성리더 양성프로그램 사업의 활동내용과 세계한민족네트워크가 나아가야 할 방향에 대해 토론한다. 미국동부, 미국서부, 캐나다, 과테말라, 중국, 아르헨티나, 인도네시아, 호주, 홍콩, 일본, 터키, 이집트, 독일, 러시아, 필리핀, 뉴질랜드, 브라질, 스페인, 말레이시아, 프랑스 등 20개 지역담당관이 참여한다. 공식 개막식이 열리는 27일 오전 10시 구미시 호텔금오산에서는 조벽 동국대 석좌교수가 일· 가정 양립과 글로벌 파트너십 강화를 주제로 기조연설을 한다. 또한 베스트셀러 미래에서 온 편지의 저자이자 지식나눔콘서트 아이러브의 강사인 정현경 뉴욕 유니언 신학대학 교수가 생애주기에 따른 경력유지 방안모색을 주제로 강연한다. 특히 일본군 위안부 피해자 특별 세션에서는 애니메이션 끝나지 않은 이야기를 시작으로 신혜수 유엔 경제적· 사회적· 문화적 권리위원회 위원의 강연, 정영태씨의 미국서부 LA지역 활동 소개, 남상구 동북아역사재단 연구위원

---

In [1]:
file_path = './common/data/lc39_data_1.hdf5'
fi = h5py.File(file_path, 'r')

In [2]:
fi.keys()

<KeysViewHDF5 ['data']>

In [15]:
group = fi['data']

In [18]:
dataset = []

for key in group.keys():
    dataset.append(group[key])

In [19]:
len(dataset)

1

In [20]:
len(dataset[0])

14548850

In [25]:
class Sampler_crawled_data(object):
    def __init__(self, group, seed=None):
        self.group = group
        self.dataset = []

        for key in group.keys():
            self.dataset.append(group[key])
        
        self.rs = np.random.RandomState(seed=seed)
        
    def sample(self, length):
        index = self.rs.randint(0, len(self.dataset[0]) - length + 1)
        return self.dataset[0][index:length+index]

In [26]:
sampler = Sampler_crawled_data(fi['data'])
sampler.sample(10)

array([ 35332,  48502,  66682,  16336,  22935, 100571,  54917, 100988,
        82571,  65726])